In [37]:
import pandas as pd
import numpy as np
import json
import matplotlib.cm as colormap
import folium
from sklearn.cluster import DBSCAN
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

 # récupérer vos données sur google takeout
 https://takeout.google.com/settings/takeout

In [38]:
json_string=open("./Takeout/Historique des positions/Historique des positions.json").read()
locations=pd.DataFrame(json.loads(json_string)["locations"])

In [39]:
locations.shape

(803222, 9)

In [40]:
locations

,accuracy,activity,altitude,heading,latitudeE7,longitudeE7,timestampMs,velocity,verticalAccuracy
0,699,"[{'timestampMs': '1549369747339', 'activity': ...",NaN,NaN,488350112,25797065,1549369675626,NaN,NaN
1,699,"[{'timestampMs': '1549369623968', 'activity': ...",NaN,NaN,488350112,25797065,1549369615542,NaN,NaN
2,699,"[{'timestampMs': '1549369505851', 'activity': ...",NaN,NaN,488350112,25797065,1549369550218,NaN,NaN
3,899,"[{'timestampMs': '1549369416422', 'activity': ...",NaN,NaN,488349797,25811148,1549369430201,NaN,NaN
4,899,"[{'timestampMs': '1549369203530', 'activity': ...",NaN,NaN,488349797,25811148,1549369310118,NaN,NaN
5,899,"[{'timestampMs': '1549369097076', 'activity': ...",NaN,NaN,488349797,25811148,1549369158454,NaN,NaN
6,899,NaN,NaN,NaN,488349797,25811148,1549369007443,NaN,NaN
7,899,"[{'timestampMs': '1549368916435', 'activity': ...",NaN,NaN,488349797,25811148,1549368887433,NaN,NaN
8,899,"[{'timestampMs': '1549368737650', 'activity': ...",NaN,NaN,488349797,25811148,1549368767422,NaN,NaN
9,899,NaN,NaN,NaN,488349797,25811148,1549368647412,NaN,NaN


In [41]:
Xt = locations.loc[:np.min([100000,locations.shape[0]]),]
Xt = Xt.loc[Xt["accuracy"]<250,:]

# Carte d'un échantillons de positions

In [42]:
m = folium.Map(
    location=[48.83, 2.58],
    tiles='Stamen Toner',
    zoom_start=13
)

In [43]:
for i,p in Xt.sample(100).iterrows():
    folium.Circle(
        radius=100,
        location=[p["latitudeE7"]/10000000, p["longitudeE7"]/10000000],
        popup='The Waterfront',
        color='crimson',
        fill=False,
    ).add_to(m)


In [44]:
m

# Recherche de clusters, hot-spot avec db-scan

In [45]:
db = DBSCAN(eps=15000,min_samples=50)

In [46]:
Xtlatlong = Xt[["latitudeE7","longitudeE7"]]
db.fit(Xtlatlong)

DBSCAN(algorithm='auto', eps=15000, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=50, n_jobs=None, p=None)

In [47]:
# combien de clusters
db.labels_.max()

37

In [48]:
# nombre de point dans un cluster
np.sum(db.labels_==3)

540

In [49]:
# calculer le centre d'un cluster
Xtlatlong.loc[db.labels_==3,].mean()/10000000

latitudeE7     48.870477
longitudeE7     2.782902
dtype: float64

In [50]:
ts = Xt.loc[db.labels_==3,"timestampMs"].apply(lambda v: int(v)/1000).mean()
datetime.utcfromtimestamp(ts).strftime('%d/%m/%Y %H:%M:%S')

'09/09/2018 15:17:32'

In [51]:
#pour convertir une colueur au foramt hexa
def tohex(col):
    rgb = tuple(int(v*255) for v in col[:3])
    return u"#{0:02x}{1:02x}{2:02x}".format(rgb[0],rgb[1],rgb[2])



In [58]:
mdbs = folium.Map(
    location=[48.83, 2.58],
    tiles='cartodbpositron',
    zoom_start=13
)
for l in range(db.labels_.max()):
    center = Xtlatlong.loc[db.labels_==l,].mean()/10000000
    ts = Xt.loc[db.labels_==l,"timestampMs"].apply(lambda v: int(v)/1000)
    time_center = datetime.utcfromtimestamp(ts.mean()).strftime('%d/%m/%Y %H:%M:%S')
    time_min = datetime.utcfromtimestamp(ts.min()).strftime('%d/%m/%Y')
    time_max = datetime.utcfromtimestamp(ts.max()).strftime('%d/%m/%Y')
    folium.CircleMarker(
        radius= np.sqrt(float(np.sum(db.labels_==l)))/10,
        color = tohex(colormap.Set1(l/db.labels_.max())),
        fill_color = tohex(colormap.Set1(l/db.labels_.max())),
        fill_opacity = 0.2,
        popup = "<h4> Cluster N° "+ str(l) +"</h4>"+str(np.sum(db.labels_==l)) + " points,</br> au alentour du "+ time_center + "</br>sur la période ["+ time_min+","+time_max +"]",
        location=center
    ).add_to(mdbs)
    

In [59]:
mdbs

# Question
Mettre en ouvre db scan sur la variable 'timestampMs' pour chaque clusteur de lieu afin d'extraire les différentes périodes pendant lesquelles ce lieu a été fréquenté. Le prendre en compte lors de l'affichage du pop up.